In [1]:
import os
import re
import sys
import json
import time
from tqdm.notebook import tqdm
from datasets import load_dataset

/Users/sanghapark/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
import torch
import random
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt

## Load Data

In [3]:
paper_data = load_dataset('metamong1/summarization_paper', 
    use_auth_token='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')

Reusing dataset paper_summarization (/Users/sanghapark/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/1.0.0/2c2f632798c1ebeb6968c73643f9e696287f305dbaa6a8581c2b2a592d3b63a0)
100%|██████████| 2/2 [00:00<00:00, 107.39it/s]


In [4]:
paper_data

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 73640
    })
    validation: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 18411
    })
})

In [5]:
train_data = paper_data['train']

In [6]:
train_data[0]

{'doc_id': 'A201008192682',
 'title': '횡적등방성 매질에서 중합전 역시간 구조보정',
 'text': '역시간 구조보정은 음원영역 파동장 외삽과 수진기영역 파동장 외삽의 상호상관으로 지층구조를 영상화하는 방법으로 복잡한 등방성 매질 층서구조를 영상화하는데 주로 이용된다. 그러나 일반적으로 지구내부 지층구조는 이방성 특성을 지니고 있으므로 이방성을 고려한 구조보정 기술이 필요하다. 여기에서는 편미분 파동장과 음원모음의 내적에 의한 알고리즘과 가상음원과 역전파 파동장과의 내적에 의한 알고리즘을 이용하여 횡적등방성 매질에서 역시간 구조보정 기술을 개발하고자 하였다. 단순 이방성 지층모델에 대한 수치모형실험 결과, 두 가지 방법에 의한 지층단면도 영상은 거의 차이가 없어 가상음원과 역전파 파동장과의 내적으로 구조보정을 실시하는 것이 효과적임을 알 수 있었다. 수평적으로 속도가 변하는 이방성 매질 지층구조에서 편미분 파동장을 구하지 않고 영상화 할 수 있음을 알 수 있었다.',
 'doc_type': '논문',
 'file': '논문요약_0206_0.json'}

## Extract Data

In [7]:
train_docs = [data['text'] for data in train_data]
train_titles = [data['title'] for data in train_data]

In [8]:
short_title = [title for title in train_titles if len(title) <= 10]
print('Size of Title which size is 1 : %d' %len(short_title))

Size of Title which size is 1 : 377


In [9]:
chn_chars = range(ord('一'), ord('鿕')+1)

In [10]:
def check_sep_title(title) :
    chars = re.sub('\s+' , '', title)
    sep_chars = [ch for ch in chars if ord(ch) in chn_chars] 
    sep_rate = len(sep_chars) / len(chars)

    return True if sep_rate >= 0.1 else False     

In [11]:
sep_titles = [title for title in tqdm(train_titles) if check_sep_title(title)]

  0%|          | 0/73640 [00:00<?, ?it/s]

In [12]:
print('Size of data which has to be checked : %d' %len(sep_titles))

Size of data which has to be checked : 3325


In [13]:
sep_titles[:10]

['陶厓 洪錫謨의 금강산 유기, 『艮觀錄』 一考',
 '규범의 근거로서 친친(親親) 존존(尊尊)의 정당화 문제',
 '한국합기도 術氣의 종류와 형태에 관한 小考',
 '‘女性嫌惡’의 文學文化史(Ⅲ) - 1900년대 文學文化의 ‘호모소셜리티’와 女性嫌惡 -',
 '경찰권 발동의 근거로서 ‘위험’의 개념과 양태 -獨逸 警察法學의 論議狀況을 참고하여-',
 '여말선초 紅牌·白牌 양식의 변화와 의의',
 '『총쇄록(叢瑣錄)』자료 해제 및 교육적 가치',
 '유커(游客)와 싼커(散客)－ 탈식민적 아시아의 정동지리학',
 '淸陰 金尙憲의 遊仙詩 小考',
 '北魏前期 胡語 사용 현상과 그 배경']

## Crawling

In [58]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

### driver 준비

In [59]:
chrome_path = '/Users/sanghapark/Desktop/Project/chromedriver'
driver = webdriver.Chrome(chrome_path)

<ipython-input-59-8d257ee82e31>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


### selenium을 통한 Web 접속

In [60]:
title = '淸陰 金尙憲의 遊仙詩 小考'

In [61]:
driver.implicitly_wait(1.5)
driver.get('https://academic.naver.com/')

In [62]:
time.sleep(0.5) ## 0.5초

element = driver.find_element_by_name('query')
element.clear()

element.send_keys(title)
element.submit()

<ipython-input-62-d6b45ebb4143>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_name('query')


### 목표로 하는 데이터 추출

In [63]:
cur_url = driver.current_url

response = requests.get(cur_url)
bs = BeautifulSoup(response.content, 'html.parser')

In [64]:
div_info = bs.findAll('div', {'class' , 'ui_listing_info'})

div_info[0].find('a').text

'청음 김상헌의 유선시 소고 (淸陰 金尙憲의 遊仙詩 小考)'